# Query WikiData to get Biomedical Entities

We will get the nodes (and later some edges) for our biomedical graph from WikiData

## Tues meet w/ Andrew
- Research Notes
- Run two codes in tandem, see how far you get with 02 of rephetio (similar or different?)

## To Do 
1. Is the script reproducible?
2. Automate reproduced script.
3. Does the script need improvements, now that's been reproduced?
4. Automate improved script.
5. Compare reproduced vs improved.
6. Test with other algorithms besides Rephetio, compare again with reproduced + improved.
7. Create web interface.

- Include mechanisms such that libraries are transferrable, no matter who does it
- Fix nodes where there's a time out (limit)
- Check and include any differing nodes (for improved section)

In [1]:
import pandas as pd
from pathlib import Path

# 'ModuleNotFoundError' for both lines below
## Solution: pip install git+https://github.com/mmayers12/data_tools
### https://github.com/mmayers12/data_tools (fyi also a data_tools in pip, different)
#### so far, these work okay (there isn't a conflict)

from data_tools.df_processing import char_combine_iter 
from data_tools.wiki import node_query_pipeline

# New line recommended by notebook
from tqdm.autonotebook import tqdm 

/Users/sulhasan/opt/anaconda3/lib/python3.8/site-packages/data_tools/df_processing.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
nodes = []

# Diseases

In [3]:
q = """SELECT DISTINCT ?disease ?diseaseLabel ?umlscui ?snomed_ct ?doid ?mesh ?mondo ?omim ?orpha
        WHERE {

          # Initial typing for Disease 
          # Either instance of Disease of has a Disease Ontology ID
          {?disease wdt:P31 wd:Q12136}UNION{?disease wdt:P699 ?doid}.

          OPTIONAL {?disease wdt:P2892 ?umlscui .}
          OPTIONAL {?disease wdt:P5806 ?snomed_ct. }
          OPTIONAL {?disease wdt:P699 ?doid. }
          OPTIONAL {?disease wdt:P486 ?mesh. }
          OPTIONAL {?disease wdt:P5270 ?mondo. }
          OPTIONAL {?disease wdt:P492 ?omim. }
          OPTIONAL {?disease wdt:P1550 ?orpha. }

          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }
        }"""

In [4]:
dis_curi_map = {'umlscui': 'UMLS', 'snomed_ct': 'SNOMED', 'mesh': 'MESH', 
                'doid': 'DOID', 'mondo': 'MONDO', 'omim': 'OMIM', 'orpha': 'ORPHA'}

res = node_query_pipeline(q, dis_curi_map, 'disease')
# what's happening in the 'node_query_pipeline()' function that's outputting format?
nodes.append(res)
nodes[0].head()

,id,name,label,xrefs
0,Q1001150,fibrillation,Disease,UMLS:C0232197
1,Q100165995,acute pulmonary hypertension,Disease,
2,Q1002195,autosomal recessive limb-girdle muscular dystr...,Disease,DOID:DOID:0110297|MONDO:MONDO:0012248|OMIM:609...
3,Q1003534,bulbar syndrome,Disease,
4,Q1004647,bullous pemphigoid,Disease,DOID:DOID:8506|MESH:D010391|MONDO:MONDO:001908...


# Compounds

In [5]:
q = """SELECT DISTINCT ?compound ?compoundLabel ?kegg_drug ?chebi ?drugbank_id ?umlscui ?chembl_id ?unii ?ikey ?pubchem_cid ?rxnorm ?mesh_supplemental_record_ui ?mesh_descriptor_ui
        WHERE {

          # Initial typing for Compound
          ?compound wdt:P31 wd:Q11173 .
          # Give me all Wikidata items where the item is an instance of a chemical compound

        # Whatever item up there may optionally have the following identifier + variable
          OPTIONAL { ?compound wdt:P665 ?kegg_drug .}
          OPTIONAL { ?compound wdt:P683 ?chebi .}
          OPTIONAL { ?compound wdt:P715 ?drugbank_id .}
          OPTIONAL { ?compound wdt:P2892 ?umlscui .}
          OPTIONAL { ?compound wdt:P592 ?chembl_id .}
          OPTIONAL { ?compound wdt:P652 ?unii .}
          OPTIONAL { ?compound wdt:P3350 ?ikey .}
          OPTIONAL { ?compound wdt:P662 ?pubchem_cid .}
          OPTIONAL { ?compound wdt:P3345 ?rxnorm .}
          OPTIONAL { ?compound wdt:P6680 ?mesh_supplemental_record_ui .}
          OPTIONAL { ?compound wdt:P486 ?mesh_descriptor_ui .}

          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }
        }
        
        limit 150000""" # limit needed here, fix later (266348 when Mike did it, but still less)
# max can be 150000 (changed from 200 to 150000)

1201261 now

SELECT (COUNT (DISTINCT ?compound) AS ?count)
                        WHERE {

          # Initial typing for Compound
          ?compound wdt:P31 wd:Q11173 .

           OPTIONAL { ?compound wdt:P665 ?kegg_drug .}
           OPTIONAL { ?compound wdt:P683 ?chebi .}
           OPTIONAL { ?compound wdt:P715 ?drugbank_id .}
           OPTIONAL { ?compound wdt:P2892 ?umlscui .}
           OPTIONAL { ?compound wdt:P592 ?chembl_id .}
           OPTIONAL { ?compound wdt:P652 ?unii .}
           OPTIONAL { ?compound wdt:P3350 ?ikey .}
           OPTIONAL { ?compound wdt:P662 ?pubchem_cid .}
           OPTIONAL { ?compound wdt:P3345 ?rxnorm .}
           OPTIONAL { ?compound wdt:P6680 ?mesh_supplemental_record_ui .}
           OPTIONAL { ?compound wdt:P486 ?mesh_descriptor_ui .}

          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }
        }

In [6]:
chem_curi_map = {'unii': 'UNII', 
    'rxnorm': 'RxCUI', 
    'drugbank_id': 'DB', 
    'umlscui': 'UMLS', 
    'chebi': 'CHEBI', 
    'chembl_id': 'CHEMBL',
    'kegg_drug': 'KEGG', 
    'ikey': 'IKEY', 
    'pubchem_cid': 'PCID', 
    'mesh_supplemental_record_ui': 'MESH', 
    'mesh_descriptor_ui': 'MESH'}

res = node_query_pipeline(q, chem_curi_map, 'compound')
nodes.append(res)
nodes[1].head()

# JSONDecodeError is due to the time it takes
## Solution: Limit above (temporary fix) 

,id,name,label,xrefs
0,Q1002152,fluoroiodomethane,Compound,PCID:13981373
1,Q1002165,fluticasone,Compound,CHEBI:5134|CHEMBL:CHEMBL1201396|KEGG:C07815|KE...
2,Q1003185,fosfestrol,Compound,CHEBI:4532|CHEMBL:CHEMBL1200598|KEGG:C08145|KE...
3,Q1003526,(S)-(+)-bulbocapnine,Compound,CHEBI:3211|CHEMBL:CHEMBL157912|KEGG:C09367|PCI...
4,Q1007138,gabaculine,Compound,CHEBI:29585|CHEMBL:CHEMBL1394922|PCID:3445


# Phenotype

In [7]:
q = """SELECT DISTINCT ?phenotype ?phenotypeLabel ?hpo ?mesh ?omim ?snomed
        WHERE {

          # Initial typing for phenotype
          {?phenotype wdt:P31 wd:Q169872.}UNION{?phenotype wdt:P3841 ?hpo}

          # Xrefs associated with phenotypes
          OPTIONAL {?phenotype wdt:P3841 ?hpo .}
          OPTIONAL {?phenotype wdt:P486 ?mesh . }
          OPTIONAL {?phenotype wdt:P492 ?omim . }
          OPTIONAL {?phenotype wdt:P5806 ?snomed . }

          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }
        }"""

In [8]:
res = node_query_pipeline(q, {'mesh': 'MESH', 'omim': 'OMIM', 'hpo':'HP', 'snomed': 'SNOMED'}, 'phenotype')
nodes.append(res)
nodes[2].head()

,id,name,label,xrefs
0,Q1016605,Burkitt lymphoma,Phenotype,HP:0030080|MESH:D002051|OMIM:113970
1,Q101971,wart,Phenotype,HP:0200043|MESH:D014860
2,Q102187007,abnormal bowel sound,Phenotype,HP:0030142
3,Q102187028,abdominal symptom,Phenotype,HP:0011458
4,Q102293266,calcium oxalate nephrolithiasis,Phenotype,HP:0008672


# Gene

Genes are too numerous and will require filtering to a single taxon in order for the query to finish successfully.

For now we will only extract human genes, but in the future we will do the same for infectious taxa.

In [9]:
q = """SELECT DISTINCT ?gene ?geneLabel ?entrez ?symbol ?hgnc ?omim ?ensembl
        WHERE {{

          # Initial typing for Gene
          ?gene wdt:P31 wd:Q7187.
          ?gene wdt:P703 wd:{tax}.

          OPTIONAL{{?gene wdt:P351 ?entrez .}}
          OPTIONAL{{?gene wdt:P353 ?symbol .}}
          OPTIONAL{{?gene wdt:P354 ?hgnc .}}
          OPTIONAL{{?gene wdt:P492 ?omim .}}
          OPTIONAL{{?gene wdt:P594 ?ensembl .}}

          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }}
        }}"""

human_tax_wd_id = 'Q15978631' 
q = q.format(tax=human_tax_wd_id)

In [10]:
gene_curi_map = {'entrez': 'NCBIGene', 'symbol': 'SYM', 'hgnc':'HGNC', 'omim':'OMIM', 'ensembl':'ENSG'}
res = node_query_pipeline(q, gene_curi_map, 'gene')
nodes.append(res)
nodes[3].head()

,id,name,label,xrefs
0,Q1022703,ZMPSTE24,Gene,ENSG:ENSG00000084073|HGNC:12877|NCBIGene:10269...
1,Q106030625,MAP2K5-DT,Gene,HGNC:55261|NCBIGene:118732298|SYM:MAP2K5-DT
2,Q106030627,CD2AP-DT,Gene,HGNC:55263|NCBIGene:118732299|SYM:CD2AP-DT
3,Q106030628,ERBIN-DT,Gene,HGNC:55270|NCBIGene:118732300|SYM:ERBIN-DT
4,Q106030629,PPP2CA-DT,Gene,HGNC:55266|NCBIGene:118732304|SYM:PPP2CA-DT


# Protein

In [11]:
q = """SELECT DISTINCT ?protein ?proteinLabel ?uniprot
        WHERE {{

          # Initial typing for Protein
          ?protein wdt:P31 wd:Q8054.
          ?protein wdt:P703 wd:{tax}.

          OPTIONAL{{?protein wdt:P352 ?uniprot .}}
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }}
        }}"""
q = q.format(tax=human_tax_wd_id)

In [12]:
res = node_query_pipeline(q, {'uniprot':'UniProt'}, 'protein')
nodes.append(res)
nodes[4].head()

,id,name,label,xrefs
0,Q1024612,C-X-C motif chemokine receptor 5,Protein,UniProt:P32302
1,Q1032902,"Mucin 16, cell surface associated",Protein,UniProt:Q8WXI7
2,Q105362742,Neurotensin/neuromedin N,Protein,UniProt:P30990
3,Q105412156,prepro-GRP,Protein,UniProt:P07492
4,Q1056532,CD44 molecule (Indian blood group),Protein,UniProt:P16070


# Pathway

In [13]:
q = """SELECT DISTINCT ?pathway ?pathwayLabel ?react ?wpid
        WHERE {

          # Initial typing for Pathway
          ?pathway wdt:P31 wd:Q4915012 .

          OPTIONAL{?pathway wdt:P3937 ?react .}
          OPTIONAL{?pathway wdt:P2410 ?wpid .}

          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }
        }"""

In [14]:
res = node_query_pipeline(q, {'react':'REACT', 'wpid':'WP'}, 'pathway')
nodes.append(res)
nodes[5].head()

,id,name,label,xrefs
0,Q100166016,EML4 and NUDC in mitotic spindle formation,Pathway,REACT:R-HSA-9648025
1,Q100166017,Postmitotic nuclear pore complex (NPC) reforma...,Pathway,REACT:R-HSA-9615933
2,Q100166018,Sealing of the nuclear envelope (NE) by ESCRT-III,Pathway,REACT:R-HSA-9668328
3,Q100166022,Inhibition of DNA recombination at telomere,Pathway,REACT:R-HSA-9670095
4,Q100166036,Response of EIF2AK4 (GCN2) to amino acid defic...,Pathway,REACT:R-HSA-9633012


# Molecular Function

In [15]:
q = """SELECT DISTINCT ?molecular_function ?molecular_functionLabel ?goid
        WHERE {

          # Initial typing for molecular Function
          ?molecular_function wdt:P31 wd:Q14860489 .
          ?molecular_function wdt:P686 ?goid

          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }
        }"""

In [16]:
res = node_query_pipeline(q, {'goid':'GO'}, 'molecular_function')
nodes.append(res)
nodes[6].head()

,id,name,label,xrefs
0,Q1012651,ribonuclease P activity,Molecular Function,GO:0004526
1,Q13667380,metal ion binding,Molecular Function,GO:0046872
2,Q13667398,lipoprotein particle receptor binding,Molecular Function,GO:0070325
3,Q14326094,protein serine/threonine kinase activity,Molecular Function,GO:0004674
4,Q14326101,serine-type peptidase activity,Molecular Function,GO:0008236


# Biological Process

In [17]:
q = """SELECT DISTINCT ?biological_process ?biological_processLabel ?goid
        WHERE {

          # Initial typing for molecular Function
          ?biological_process wdt:P31 wd:Q2996394 .
          ?biological_process wdt:P686 ?goid

          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }
        }"""

In [18]:
res = node_query_pipeline(q, {'goid':'GO'}, 'biological_process')
nodes.append(res)
nodes[7].head()

,id,name,label,xrefs
0,Q10344426,pachytene,Biological Process,GO:0000239
1,Q1057,metabolism,Biological Process,GO:0008152
2,Q105726,urination,Biological Process,GO:0060073
3,Q1068809,cell cycle checkpoint,Biological Process,GO:0000075
4,Q10746904,intramembranous ossification,Biological Process,GO:0001957


# Cellular Component

In [19]:
q = """SELECT DISTINCT ?cellular_component ?cellular_componentLabel ?goid
    WHERE {

      # Initial typing for Cellular Component
      ?cellular_component wdt:P31 wd:Q5058355 .
      ?cellular_component wdt:P686 ?goid

      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }
    }"""

In [20]:
res = node_query_pipeline(q, {'goid':'GO'}, 'cellular_component')
nodes.append(res)
nodes[8].head()

,id,name,label,xrefs
0,Q1051872,caveola,Cellular Component,GO:0005901
1,Q1053832,tight junction,Cellular Component,GO:0070160
2,Q1065756,melanosome,Cellular Component,GO:0042470
3,Q1134046,plasmodesma,Cellular Component,GO:0009506
4,Q1136436,costamere,Cellular Component,GO:0043034


# Anatomy

In [21]:
q = """SELECT DISTINCT ?anatomy ?anatomyLabel ?uberon ?mesh
        WHERE {

          # Anatomical Strucutres
          ?anatomy wdt:P1554 ?uberon
          
          OPTIONAL{?anatomy wdt:P486 ?mesh .}

          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }
        }"""

In [22]:
res = node_query_pipeline(q, {'uberon':'UBERON', 'mesh': 'MESH'}, 'anatomy')
nodes.append(res)
nodes[9].head()

,id,name,label,xrefs
0,Q1001337,mesencephalic nucleus of trigeminal nerve,Anatomy,UBERON:0001718
1,Q1002789,posterior ethmoidal foramen,Anatomy,UBERON:0018654
2,Q1003805,Nucleus ambiguus,Anatomy,UBERON:0001719
3,Q101004,aorta,Anatomy,MESH:D001011|UBERON:0000947
4,Q102277188,anatomical projection,Anatomy,UBERON:0004529


# Put them all together

In [23]:
nodes = pd.concat(nodes, sort=False, ignore_index=True)
len(nodes)

301595

In [24]:
nodes['id'].nunique()

300440

In [25]:
nodes[nodes['id'].duplicated(keep=False)].sort_values('id').head(50)

,id,name,label,xrefs
15,Q1016605,Burkitt lymphoma,Disease,DOID:DOID:8584|MESH:D002051|OMIM:113970|ORPHA:...
165242,Q1016605,Burkitt lymphoma,Phenotype,HP:0030080|MESH:D002051|OMIM:113970
165243,Q101971,wart,Phenotype,HP:0200043|MESH:D014860
20,Q101971,wart,Disease,MESH:D014860|UMLS:C3665596
66,Q102293266,calcium oxalate nephrolithiasis,Disease,DOID:DOID:0080652
165246,Q102293266,calcium oxalate nephrolithiasis,Phenotype,HP:0008672
876,Q1027995,pyloric stenosis,Disease,DOID:DOID:12639|MESH:D011707|UMLS:C0034194
165247,Q1027995,pyloric stenosis,Phenotype,HP:0002021|MESH:D011707
165248,Q10282075,gingival fibromatosis,Phenotype,HP:0000169|MESH:D005351|OMIM:135300|OMIM:60554...
877,Q10282075,gingival fibromatosis,Disease,DOID:DOID:0060466|MESH:D005351|MONDO:MONDO:001...


In [26]:
nodes[nodes['id'].duplicated(keep=False)].sort_values('id').tail(50)

,id,name,label,xrefs
16442,Q947813,encephalomalacia,Disease,DOID:DOID:2034|MESH:D004678|MONDO:MONDO:000674...
167207,Q947813,encephalomalacia,Phenotype,HP:0040197|MESH:D004678
167208,Q950591,scotoma,Phenotype,HP:0000575|MESH:D012607
16447,Q950591,scotoma,Disease,DOID:DOID:9335|MESH:D012607|MONDO:MONDO:000475...
167209,Q950838,germinoma,Phenotype,HP:0100620|MESH:D018237|SNOMED:28307001
16448,Q950838,germinoma,Disease,DOID:DOID:3304|MESH:D018237|MONDO:MONDO:000259...
167210,Q954625,myelodysplastic syndrome,Phenotype,HP:0002863|MESH:D009190|OMIM:614286|SNOMED:109...
16452,Q954625,myelodysplastic syndrome,Disease,DOID:DOID:0050908|MESH:D009190|MONDO:MONDO:001...
167211,Q954674,iron deficiency anemia,Phenotype,HP:0001891|MESH:D018798
16453,Q954674,iron deficiency anemia,Disease,DOID:DOID:11758|MESH:D018798|MONDO:MONDO:00013...


In [27]:
nodes['label'].value_counts()

Compound              148710
Gene                   59061
Biological Process     28674
Protein                25476
Disease                16532
Molecular Function     11029
Cellular Component      4155
Pathway                 3389
Anatomy                 2567
Phenotype               2002
Name: label, dtype: int64

## Save

In [28]:
out_dir = Path('../results/')

# Make the output directory if doesn't already exist
out_dir.mkdir(parents=True, exist_ok=True)

nodes.to_csv(out_dir.joinpath('01a_nodes.csv'), index=False)

## edit 'pipeline' folder to be results?